In [1]:
import pandas as pd
import os

In [12]:
df = pd.read_csv('Dataset.csv', header=None)

# --- Create the new headers ---

# Get the top header row (row 0) and fill 'Question 1' into the blank cells next to it.
header_top = df.iloc[0].ffill()

header_bottom = df.iloc[1]

new_cols = []
for top, bottom in zip(header_top, header_bottom):
    # If the bottom row is blank (NaN) or a repeat, just use the top row.
    if pd.isna(bottom) or top == bottom:
        new_cols.append(top)
    # Otherwise, combine them.
    else:
        new_cols.append(f'{top} - {bottom}')

df.columns = new_cols
df.columns

Index(['Respondent ID', 'Start Date', 'End Date', 'Email Address',
       'First Name', 'Last Name', 'Custom Data 1',
       'Identify which division you work in. - Response',
       'Which of the following best describes your position level? - Response',
       'Which generation are you apart of? - Response',
       'Please select the gender in which you identify. - Response',
       'Which duration range best aligns with your tenure at your company? - Response',
       'Which of the following best describes your employment type? - Response',
       'Question 1 - Response', 'Question 2 - Response',
       'Question 3 - Open-Ended Response', 'Question 4 - Response',
       'Question 4 - Other (please specify)', 'Question 5 - Response 1',
       'Question 5 - Response 2', 'Question 5 - Response 3',
       'Question 5 - Response 4', 'Question 5 - Response 5',
       'Question 5 - Response 6', 'Question 6 - Response 1',
       'Question 6 - Response 2', 'Question 6 - Response 3',
       '

In [14]:
# Drop the first two rows (which were the old headers)
# and reset the index
df = df.iloc[2:].reset_index(drop=True)

# --- Unpivot the data ---

# First, define your 'ID' columns. These are the columns
# that identify a respondent and should NOT be unpivoted.
# Get these names from the 'new_cols' list you just made.
id_vars = [
    'Respondent ID', 'Start Date', 'End Date', 'Email Address',
       'First Name', 'Last Name', 'Custom Data 1',
       'Identify which division you work in. - Response',
       'Which of the following best describes your position level? - Response',
       'Which generation are you apart of? - Response',
       'Please select the gender in which you identify. - Response',
       'Which duration range best aligns with your tenure at your company? - Response',
       'Which of the following best describes your employment type? - Response'
]

# Melt the DataFrame
# 'id_vars' are the columns to keep
# 'var_name' is the name of the new column that holds the question names
# 'value_name' is the name of the new column that holds the answers
melted_df = df.melt(
    id_vars=id_vars, 
    var_name='Question - Subquestion', 
    value_name='Answer'
)

# Show the first few rows of the new, long-format data
melted_df.shape

(16856, 15)

In [15]:
melted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16856 entries, 0 to 16855
Data columns (total 15 columns):
 #   Column                                                                         Non-Null Count  Dtype 
---  ------                                                                         --------------  ----- 
 0   Respondent ID                                                                  16856 non-null  object
 1   Start Date                                                                     16856 non-null  object
 2   End Date                                                                       16856 non-null  object
 3   Email Address                                                                  0 non-null      object
 4   First Name                                                                     0 non-null      object
 5   Last Name                                                                      0 non-null      object
 6   Custom Data 1                 

In [21]:
melted_df = melted_df.rename(columns=
    {
       'Question - Subquestion': 'question_subquestion',
       'Answer': 'answer' 
    }
)
melted_df


,respondent_id,start_date,end_date,email_address,first_name,last_name,Custom Data 1,division,position_level,generation,gender,tenure_duration,employment_type,question_subquestion,answer
0,4044163394,1/22/2021 6:35,1/22/2021 6:47,NaN,NaN,NaN,NaN,Infrastructure,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,Answer 5
1,5535865599,1/21/2021 21:29,1/21/2021 21:40,NaN,NaN,NaN,NaN,Infrastructure,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1 - Response,Answer 1
2,3356802928,1/21/2021 17:26,1/21/2021 17:44,NaN,NaN,NaN,NaN,Port Operations,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1 - Response,NaN
3,3399511781,1/21/2021 11:40,1/21/2021 11:51,NaN,NaN,NaN,NaN,Infrastructure,Staff,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,Answer 8
4,9860597462,1/21/2021 10:01,1/21/2021 10:48,NaN,NaN,NaN,NaN,Infrastructure,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 1 - Response,Answer 8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16851,7940065082,1/11/2021 6:19,1/11/2021 6:32,NaN,NaN,NaN,NaN,Infrastructure,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30 - Response 3,Answer 8
16852,5157705612,1/11/2021 6:19,1/11/2021 6:33,NaN,NaN,NaN,NaN,Finance,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30 - Response 3,Answer 6
16853,9920755555,1/11/2021 6:18,1/11/2021 6:27,NaN,NaN,NaN,NaN,Port Operations,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN
16854,6638341389,1/11/2021 6:19,1/11/2021 6:33,NaN,NaN,NaN,NaN,Infrastructure,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN


In [24]:
melted_df.columns

Index(['respondent_id', 'start_date', 'end_date', 'email_address',
       'first_name', 'last_name', 'Custom Data 1', 'division',
       'position_level', 'generation', 'gender', 'tenure_duration',
       'employment_type', 'question_subquestion', 'answer'],
      dtype='object')

In [34]:
columns_to_drop = ['email_address',
       'first_name', 'last_name', 'Custom Data 1','start_date', 'end_date',]
melted_df_col = melted_df.drop(columns=columns_to_drop)
melted_df_col

,respondent_id,division,position_level,generation,gender,tenure_duration,employment_type,question_subquestion,answer
0,4044163394,Infrastructure,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,Answer 5
1,5535865599,Infrastructure,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1 - Response,Answer 1
2,3356802928,Port Operations,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1 - Response,NaN
3,3399511781,Infrastructure,Staff,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,Answer 8
4,9860597462,Infrastructure,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 1 - Response,Answer 8
...,...,...,...,...,...,...,...,...,...
16851,7940065082,Infrastructure,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30 - Response 3,Answer 8
16852,5157705612,Finance,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30 - Response 3,Answer 6
16853,9920755555,Port Operations,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN
16854,6638341389,Infrastructure,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN


In [35]:
melted_df_col.isna().sum()

respondent_id              0
division                  86
position_level           172
generation                86
gender                    86
tenure_duration            0
employment_type            0
question_subquestion       0
answer                  7294
dtype: int64

In [36]:
melted_df_col.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16856 entries, 0 to 16855
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   respondent_id         16856 non-null  object
 1   division              16770 non-null  object
 2   position_level        16684 non-null  object
 3   generation            16770 non-null  object
 4   gender                16770 non-null  object
 5   tenure_duration       16856 non-null  object
 6   employment_type       16856 non-null  object
 7   question_subquestion  16856 non-null  object
 8   answer                9562 non-null   object
dtypes: object(9)
memory usage: 1.3+ MB


In [39]:
df_clean = melted_df_col.dropna(subset=['answer'], inplace=False)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9562 entries, 0 to 16852
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   respondent_id         9562 non-null   object
 1   division              9518 non-null   object
 2   position_level        9468 non-null   object
 3   generation            9514 non-null   object
 4   gender                9512 non-null   object
 5   tenure_duration       9562 non-null   object
 6   employment_type       9562 non-null   object
 7   question_subquestion  9562 non-null   object
 8   answer                9562 non-null   object
dtypes: object(9)
memory usage: 747.0+ KB


In [43]:
question_import = pd.read_excel('Data - Survey Monkey Output Edited.xlsx', sheet_name='Question')
question_import

,Raw Question,Raw Subquestion,Question,Subquestion,Question + Subquestion
0,Respondent ID,NaN,Respondent ID,NaN,Respondent ID
1,Start Date,NaN,Start Date,NaN,Start Date
2,End Date,NaN,End Date,NaN,End Date
3,Email Address,NaN,Email Address,NaN,Email Address
4,First Name,NaN,First Name,NaN,First Name
...,...,...,...,...,...
95,NaN,Response 13,Question 29,Response 13,Question 29 - Response 13
96,NaN,Response 14,Question 29,Response 14,Question 29 - Response 14
97,Question 30,Response 1,Question 30,Response 1,Question 30 - Response 1
98,NaN,Response 2,Question 30,Response 2,Question 30 - Response 2


In [46]:
questions = question_import.copy()
questions.drop(columns=['Raw Question', 'Raw Subquestion', 'Subquestion'], inplace=True)
questions

,Question,Question + Subquestion
0,Respondent ID,Respondent ID
1,Start Date,Start Date
2,End Date,End Date
3,Email Address,Email Address
4,First Name,First Name
...,...,...
95,Question 29,Question 29 - Response 13
96,Question 29,Question 29 - Response 14
97,Question 30,Question 30 - Response 1
98,Question 30,Question 30 - Response 2


In [63]:
dataset_merged = pd.merge(left = melted_df_col, right = questions, 
                          how = 'left', left_on = 'question_subquestion', right_on = 'Question + Subquestion')

In [66]:
#dataset_merged['answer'].dropna()
dataset_merged_subset = dataset_merged[dataset_merged['answer'].notna()]
dataset_merged_subset.groupby('Question')['respondent_id'].nunique().reset_index()
dataset_merged_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9562 entries, 0 to 16852
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   respondent_id           9562 non-null   object
 1   division                9518 non-null   object
 2   position_level          9468 non-null   object
 3   generation              9514 non-null   object
 4   gender                  9512 non-null   object
 5   tenure_duration         9562 non-null   object
 6   employment_type         9562 non-null   object
 7   question_subquestion    9562 non-null   object
 8   answer                  9562 non-null   object
 9   Question                9562 non-null   object
 10  Question + Subquestion  9562 non-null   object
dtypes: object(11)
memory usage: 896.4+ KB


In [67]:
dataset_merged_subset.groupby('Question')['respondent_id'].nunique().reset_index()

,Question,respondent_id
0,Question 1,118
1,Question 10,196
2,Question 11,162
3,Question 12,113
4,Question 13,108
5,Question 14,103
6,Question 15,112
7,Question 16,115
8,Question 17,134
9,Question 18,108


In [74]:
dataset_merged_subset.groupby(['Question + Subquestion', 'answer'])['respondent_id'].nunique().reset_index()

,Question + Subquestion,answer,respondent_id
0,Question 1 - Response,Answer 1,14
1,Question 1 - Response,Answer 2,10
2,Question 1 - Response,Answer 3,13
3,Question 1 - Response,Answer 4,16
4,Question 1 - Response,Answer 5,22
...,...,...,...
683,Question 9 - Response 4,Answer 4,16
684,Question 9 - Response 4,Answer 5,13
685,Question 9 - Response 4,Answer 6,14
686,Question 9 - Response 4,Answer 7,12


In [75]:
dataset_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16856 entries, 0 to 16855
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   respondent_id           16856 non-null  object
 1   division                16770 non-null  object
 2   position_level          16684 non-null  object
 3   generation              16770 non-null  object
 4   gender                  16770 non-null  object
 5   tenure_duration         16856 non-null  object
 6   employment_type         16856 non-null  object
 7   question_subquestion    16856 non-null  object
 8   answer                  9562 non-null   object
 9   Question                16856 non-null  object
 10  Question + Subquestion  16856 non-null  object
dtypes: object(11)
memory usage: 1.4+ MB


In [76]:
output = dataset_merged
output

,respondent_id,division,position_level,generation,gender,tenure_duration,employment_type,question_subquestion,answer,Question,Question + Subquestion
0,4044163394,Infrastructure,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,Answer 5,Question 1,Question 1 - Response
1,5535865599,Infrastructure,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1 - Response,Answer 1,Question 1,Question 1 - Response
2,3356802928,Port Operations,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1 - Response,NaN,Question 1,Question 1 - Response
3,3399511781,Infrastructure,Staff,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,Answer 8,Question 1,Question 1 - Response
4,9860597462,Infrastructure,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 1 - Response,Answer 8,Question 1,Question 1 - Response
...,...,...,...,...,...,...,...,...,...,...,...
16851,7940065082,Infrastructure,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30 - Response 3,Answer 8,Question 30,Question 30 - Response 3
16852,5157705612,Finance,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30 - Response 3,Answer 6,Question 30,Question 30 - Response 3
16853,9920755555,Port Operations,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN,Question 30,Question 30 - Response 3
16854,6638341389,Infrastructure,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN,Question 30,Question 30 - Response 3


In [78]:
output.to_excel('Dataset_clean.xlsx', index=False)